In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Message Classifier/Messages - Sheet1.csv')

In [ ]:
data

,text,label
0,Iris downtime from 7pm today,1.0
1,"IET's very own, INKHEART RECOMMENDS! 🎊🎊\n\nCha...",2.0
2,Greetings from Cisco!!\n\nWho doesn’t want bra...,2.0
3,"Please pm any senior ig you cannot join, other...",1.0
4,"Guys, we will be having SIG meet just after ML...",1.0
...,...,...
96,"We'll be having many guests today, both from N...",3.0
97,Actually this concept was used in incident pro...,2.0
98,They asked everyone to switch on and took the ...,3.0
99,In M1 videos are till instruction and data mem...,3.0


In [ ]:
data = data.dropna()

In [ ]:
data.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
data['text'] = [entry.lower() for entry in data['text']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
X_train, X_test,Y_train, Y_test = train_test_split(data['text'], data['label'], test_size=0.2)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_test_counts = count_vect.fit_transform(X_test)
X_counts = count_vect.fit_transform(data['text'])
X_train_counts.shape, X_test_counts.shape
#print(count_vect.get_feature_names())

((80, 1106), (20, 168))

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
#tf_transformer1 = TfidfVectorizer(max_features=5000)
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
tf_transformer1 = TfidfTransformer(use_idf=True).fit(X_test_counts)
X_test_tf = tf_transformer1.transform(X_test_counts)
#X_train_tf = tf_transformer.fit(X_train)
#X_test_tf = tf_transformer.fit(X_test)
X_train_tf.shape, X_test_tf.shape

((80, 1106), (20, 168))

In [ ]:
#diff model
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
#X_test_counts = count_vect.fit_transform(X_test)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [ ]:
a = clf.predict(count_vect.transform(X_test))
clf.score(count_vect.transform(X_test), y_test)
#clf.predict(count_vect.transform(["Sir is not responding to msgs, mails, call at present. So we have to wait."]))

0.76

In [ ]:
parameters = [{'gamma': [0.001, 0.003, 0.005, 0.007],
                     'C': [0.1, 1, 10, 100]}]

In [ ]:
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
svc = svm.SVC(kernel='linear', degree=3, gamma='auto')
svm_clf = GridSearchCV(svc, parameters)

In [ ]:
svm_clf.fit(X_train_tfidf,y_train)
#svm_clf.best_params_

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [0.1, 1, 10, 100],
                          'gamma': [0.001, 0.003, 0.005, 0.007]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
svm_clf.score(count_vect.transform(X_test), y_test)

0.72

In [ ]:
svm_clf.predict(count_vect.transform(["Today's Quiz cancelled"]))

array([1.])

In [ ]:
params = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
         }
MNB = GridSearchCV(MultinomialNB(), param_grid=params, n_jobs=-1, cv=2, verbose=10)

In [ ]:
MNB.fit(X_train_tfidf,y_train)
MNB.score(count_vect.transform(X_test), y_test)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.4s finished


0.72

In [ ]:
MNB.predict(count_vect.transform(["Hey there"]))

array([1.])

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
#model = GradientBoostingClassifier(n_estimators=50, learning_rate=1.0, max_depth=3, random_state=0).fit(X_train_tfidf, y_train)
params = {
    'n_estimators': [50, 100, 150],
    'max_depth': [2, 3, 4]
}
model = GridSearchCV(GradientBoostingClassifier(learning_rate=1.0), param_grid=params, n_jobs=-1, verbose=10)

In [ ]:
model.fit(X_train_tfidf, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    7.8s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=1.0,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
      

In [ ]:
model.score(count_vect.transform(X_test), y_test)

0.76

In [ ]:
model.best_params_

{'max_depth': 2, 'n_estimators': 100}

In [ ]:
model.predict(count_vect.transform(['Quiz cancelled']))

array([1.])

In [ ]:
import pickle
pickle.dump(model, open('model.pkl','wb'))

In [ ]:
from google.colab import files
files.download('model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.predict(count_vect.transform(["How are you doing"]))

array([3.])

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


parameters = {'C': [0.01, 1, 10, 100]
              }
                  
model = GridSearchCV(LinearSVC(), param_grid=parameters)
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(count_vect.transform(X_test))

# Evaluation
model.score(count_vect.transform(X_test), y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.76

In [ ]:
model.best_params_

{'C': 1}